In [2]:
import os

path = r"C:\Users\ADEKUNLE\Documents\GitHub\DAT210x\Module4"
os.chdir(path)

In [24]:
%%writefile assignment1.py
# %load assignment1.py
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime

from mpl_toolkits.mplot3d import Axes3D
from plyfile import PlyData, PlyElement


# Every 100 data samples, we save 1. If things run too
# slow, try increasing this number. If things run too fast,
# try decreasing it... =)
reduce_factor = 100


# Look pretty...
matplotlib.style.use('ggplot')


# Load up the scanned armadillo
plyfile = PlyData.read('Datasets/stanford_armadillo.ply')
armadillo = pd.DataFrame({
  'x':plyfile['vertex']['z'][::reduce_factor],
  'y':plyfile['vertex']['x'][::reduce_factor],
  'z':plyfile['vertex']['y'][::reduce_factor]
})



def do_PCA(armadillo):
  #
  # TODO: Write code to import the libraries required for PCA.
  # Then, train your PCA on the armadillo dataframe. Finally,
  # drop one dimension (reduce it down to 2D) and project the
  # armadillo down to the 2D principal component feature space.
  #
    from sklearn.decomposition import PCA
    pca = PCA(n_components = 2)
    pca.fit(armadillo)
    
    armadillo_P = pca.transform(armadillo)
  # NOTE: Be sure to RETURN your projected armadillo! 
  # (This projection is actually stored in a NumPy NDArray and
  # not a Pandas dataframe, which is something Pandas does for
  # you automatically. =)
  #
  # .. your code here ..
    return armadillo_P


def do_RandomizedPCA(armadillo):
  #
  # TODO: Write code to import the libraries required for
  # RandomizedPCA. Then, train your RandomizedPCA on the armadillo
  # dataframe. Finally, drop one dimension (reduce it down to 2D)
  # and project the armadillo down to the 2D principal component
  # feature space.
    from sklearn.decomposition import RandomizedPCA
    RandomizedPca = RandomizedPCA(n_components = 2)
    RandomizedPca.fit(armadillo)
    
    armadillo_RP = RandomizedPca.transform(armadillo)
  # NOTE: Be sure to RETURN your projected armadillo! 
  # (This projection is actually stored in a NumPy NDArray and
  # not a Pandas dataframe, which is something Pandas does for
  # you automatically. =)
  #
  # .. your code here ..
    return armadillo_RP



# Render the Original Armadillo
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d') #remember that whenever you are plotting 3D you add projection to the add_subplot
ax.set_title('Armadillo 3D')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.scatter(armadillo.x, armadillo.y, armadillo.z, c='green', marker='.', alpha=0.75)



# Time the execution of PCA 5000x
t1 = datetime.datetime.now()
for i in range(5000): pca = do_PCA(armadillo)
time_delta = datetime.datetime.now() - t1

# Render the newly transformed PCA armadillo!
if not pca is None: #intuitively saying if pca = do_PCA(armadillo) is not None
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title('PCA, build time: ' + str(time_delta))
    ax.scatter(pca[:,0], pca[:,1], c='blue', marker='.', alpha=0.75)

# Time the execution of rPCA 5000x
t1 = datetime.datetime.now()
for i in range(5000): rpca = do_RandomizedPCA(armadillo)
time_delta = datetime.datetime.now() - t1

# Render the newly transformed RandomizedPCA armadillo!
if not rpca is None:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title('RandomizedPCA, build time: ' + str(time_delta))
    ax.scatter(rpca[:,0], rpca[:,1], c='red', marker='.', alpha=0.75)


plt.show()

Overwriting assignment1.py


In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import assignment2_helper as helper
%matplotlib inline

# Look pretty...
matplotlib.style.use('ggplot')


# Do * NOT * alter this line, until instructed!
scaleFeatures = True


# TODO: Load up the dataset and remove any and all
# Rows that have a nan. You should be a pro at this
# by now ;-)
#
# .. your code here ..
kidneyData = pd.read_csv('Datasets/kidney_disease.csv')
kidneyDataNull = kidneyData.dropna(axis = 0, how='any' ).reset_index(drop = True)

# Create some color coded labels; the actual label feature
# will be removed prior to executing PCA, since it's unsupervised.
# You're only labeling by color so you can see the effects of PCA
labels = ['red' if i=='ckd' else 'green' for i in kidneyDataNull.classification]


# TODO: Use an indexer to select only the following columns:
#       ['bgr','wc','rc']
#
# .. your code here ..

selectMain = kidneyDataNull.loc[:, ['bgr', 'wc', 'rc']]

# TODO: Print out and check your dataframe's dtypes. You'll probably
# want to call 'exit()' after you print it out so you can stop the
# program's execution.
#
print(selectMain.dtypes)


# You can either take a look at the dataset webpage in the attribute info
# section: https://archive.ics.uci.edu/ml/datasets/Chronic_Kidney_Disease
# or you can actually peek through the dataframe by printing a few rows.
# What kind of data type should these three columns be? If Pandas didn't
# properly detect and convert them to that data type for you, then use
# an appropriate command to coerce these features into the right type.
#
# .. your code here ..

selectMain = selectMain.apply(lambda s: pd.to_numeric(s))
print(selectMain.dtypes)
# TODO: PCA Operates based on variance. The variable with the greatest
# variance will dominate. Go ahead and peek into your data using a
# command that will check the variance of every feature in your dataset.

print(selectMain.var())
# Print out the results. Also print out the results of running .describe
# on your dataset.
#
# Hint: If you don't see all three variables: 'bgr','wc' and 'rc', then
# you probably didn't complete the previous step properly.
#
# .. your code here ..
print(selectMain.describe())


# TODO: This method assumes your dataframe is called df. If it isn't,
# make the appropriate changes. Don't alter the code in scaleFeatures()
# just yet though!
#
# .. your code adjustment here ..
if scaleFeatures: selectMain = helper.scaleFeatures(selectMain)


# TODO: Run PCA on your dataset and reduce it to 2 components
# Ensure your PCA instance is saved in a variable called 'pca',
# and that the results of your transformation are saved in 'T'.
#
# .. your code here ..
    #import PCA from sklearn
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
pca.fit(selectMain)
T = pca.transform(selectMain)


# Plot the transformed data as a scatter plot. Recall that transforming
# the data will result in a NumPy NDArray. You can either use MatPlotLib
# to graph it directly, or you can convert it to DataFrame and have pandas
# do it for you.
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
ax.set_title('Unreduced Format of selectMain')
ax.set_xlabel('bgr')
ax.set_ylabel('rc')
ax.set_zlabel('wc')
ax.scatter(selectMain.bgr, selectMain.rc, selectMain.wc, c = 'green', marker = '.')
#

#plot reduced form of selectMain
fig = plt.figure()
ax2 = fig.add_subplot(111)
ax2.set_title('PCA reduction of Selected Features of KidneyData')
ax2.set_xlabel('Component 1')
ax2.set_ylabel('Component 2')
ax2.scatter(T[:, 0], T[:, 1], c = 'orange' , marker = 'o', alpha = 0.75)
plt.show()
# Since we've already demonstrated how to plot directly with MatPlotLib in
# Module4/assignment1.py, this time we'll convert to a Pandas Dataframe.
#
# Since we transformed via PCA, we no longer have column names. We know we
# are in P.C. space, so we'll just define the coordinates accordingly:
ax = helper.drawVectors(T, pca.components_, selectMain.columns.values, plt, scaleFeatures)
T = pd.DataFrame(T)
T.columns = ['component1', 'component2']

T.plot.scatter(x='component1', y='component2', marker='o', c=labels, alpha=0.75, ax=ax)
plt.show()




Writing assignment3.py


In [57]:
pca.components_

array([[ 0.60180443,  0.49016625, -0.63053032],
       [-0.45604187,  0.85904029,  0.23254161]])

In [ ]:
# %load assignment2_helper.py
import math
import pandas as pd
from sklearn import preprocessing

# A Note on SKLearn .transform() calls:
#
# Any time you transform your data, you lose the column header names.
# This actually makes complete sense. There are essentially two types
# of transformations,  those that change the scale of your features,
# and those that change your features entire. Changing the scale would
# be like changing centimeters to inches. Changing the features would
# be like using PCA to reduce 300 columns to 30. In either case, the
# original column's units have been altered or no longer exist, so it's
# up to you to rename your columns after ANY transformation. Due to
# this, SKLearn returns an NDArray from *transform() calls.

def scaleFeatures(df):
  # SKLearn has many different methods for doing transforming your
  # features by scaling them (this is a type of pre-processing).
  # RobustScaler, Normalizer, MinMaxScaler, MaxAbsScaler, StandardScaler...
  # http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing
  #
  # However in order to be effective at PCA, there are a few requirements
  # that must be met, and which will drive the selection of your scaler.
  # PCA required your data is standardized -- in other words it's mean is
  # equal to 0, and it has ~unit variance.
  #
  # SKLearn's regular Normalizer doesn't zero out the mean of your data,
  # it only clamps it, so it's inappropriate to use here (depending on
  # your data). MinMaxScaler and MaxAbsScaler both fail to set a unit
  # variance, so you won't be using them either. RobustScaler can work,
  # again depending on your data (watch for outliers). For these reasons
  # we're going to use the StandardScaler. Get familiar with it by visiting
  # these two websites:
  #
  # http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler
  #
  # http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler
  #


  # ---------
  # Feature scaling is the type of transformation that only changes the
  # scale and not number of features, so we'll use the original dataset
  # column names. However we'll keep in mind that the _units_ have been
    # altered:
    scaled = preprocessing.StandardScaler().fit_transform(df)
    scaled = pd.DataFrame(scaled, columns=df.columns)
    print("New Variances:\n", scaled.var())
    print("New Describe:\n", scaled.describe())
    return scaled


def drawVectors(transformed_features, components_, columns, plt, scaled):
  if not scaled:
    return plt.axes() # No cheating ;-)

  num_columns = len(columns)

  # This funtion will project your *original* feature (columns)
  # onto your principal component feature-space, so that you can
  # visualize how "important" each one was in the
  # multi-dimensional scaling
  
  # Scale the principal components by the max value in
  # the transformed set belonging to that component
  xvector = components_[0] * max(transformed_features[:,0])
  yvector = components_[1] * max(transformed_features[:,1])

  ## visualize projections

  # Sort each column by it's length. These are your *original*
  # columns, not the principal components.
  important_features = { columns[i] : math.sqrt(xvector[i]**2 + yvector[i]**2) for i in range(num_columns) }
  important_features = sorted(zip(important_features.values(), important_features.keys()), reverse=True)
  print("Features by importance:\n", important_features)

  ax = plt.axes()

  for i in range(num_columns):
    # Use an arrow to project each original feature as a
    # labeled vector on your principal component axes
    plt.arrow(0, 0, xvector[i], yvector[i], color='b', width=0.0005, head_width=0.02, alpha=0.75)
    plt.text(xvector[i]*1.2, yvector[i]*1.2, list(columns)[i], color='b', alpha=0.75)

  return ax
    

In [4]:
%%writefile assignment3.py
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import assignment2_helper as helper
%matplotlib inline

# Look pretty...
matplotlib.style.use('ggplot')


# Do * NOT * alter this line, until instructed!
scaleFeatures = True


# TODO: Load up the dataset and remove any and all
# Rows that have a nan. You should be a pro at this
# by now ;-)
#
# .. your code here ..
kidneyData = pd.read_csv('Datasets/kidney_disease.csv')
kidneyDataNull = kidneyData.dropna(axis = 0, how='any' ).reset_index(drop = True)

# Create some color coded labels; the actual label feature
# will be removed prior to executing PCA, since it's unsupervised.
# You're only labeling by color so you can see the effects of PCA
labels = ['red' if i=='ckd' else 'green' for i in kidneyDataNull.classification]


# TODO: Use an indexer to select only the following columns:
#       ['bgr','wc','rc']
#
# .. your code here ..

#selectMain = kidneyDataNull.loc[:, ['bgr', 'wc', 'rc']]

# TODO: Print out and check your dataframe's dtypes. You'll probably
# want to call 'exit()' after you print it out so you can stop the
# program's execution.
#
#print(selectMain.dtypes)


# You can either take a look at the dataset webpage in the attribute info
# section: https://archive.ics.uci.edu/ml/datasets/Chronic_Kidney_Disease
# or you can actually peek through the dataframe by printing a few rows.
# What kind of data type should these three columns be? If Pandas didn't
# properly detect and convert them to that data type for you, then use
# an appropriate command to coerce these features into the right type.
#
# .. your code here ..

#selectMain = selectMain.apply(lambda s: pd.to_numeric(s))
#print(selectMain.dtypes)
# TODO: PCA Operates based on variance. The variable with the greatest
# variance will dominate. Go ahead and peek into your data using a
# command that will check the variance of every feature in your dataset.

#print(selectMain.var())
# Print out the results. Also print out the results of running .describe
# on your dataset.
#The code here is when you drop the nominal features and run on only the numerical features
#kidneyDataDrop = kidneyData.drop(['id', 'classification', 'rbc', 'pc', 'pcc', 'ba', 'htn', 'dm',\
#                                  'cad', 'appet', 'pe', \
#                                  'ane'], axis = 1).dropna(axis = 0, how = 'any').reset_index(drop = True)
#print(kidneyDataDrop.head())

#selectMain = kidneyDataDrop.apply(lambda j: pd.to_numeric(j))

#Trying out when the nominal values are there and you encode the nominal features 
kidneyDataNominal = kidneyData.drop(['id', 'classification'], axis = 1).dropna(axis = 0, how = 'any').reset_index(drop = True)
selectMain = pd.get_dummies(kidneyDataNominal, columns= ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm',\
                                                         'cad', 'appet', 'pe', 'ane'] )
print(selectMain.dtypes)
selectMain = selectMain.apply(lambda m: pd.to_numeric(m))
# Hint: If you don't see all three variables: 'bgr','wc' and 'rc', then
# you probably didn't complete the previous step properly.
#
# .. your code here ..
print(selectMain.describe())


# TODO: This method assumes your dataframe is called df. If it isn't,
# make the appropriate changes. Don't alter the code in scaleFeatures()
# just yet though!
#
# .. your code adjustment here ..
if scaleFeatures: selectMain = helper.scaleFeatures(selectMain)


# TODO: Run PCA on your dataset and reduce it to 2 components
# Ensure your PCA instance is saved in a variable called 'pca',
# and that the results of your transformation are saved in 'T'.
#
# .. your code here ..
    #import PCA from sklearn
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
pca.fit(selectMain)
T = pca.transform(selectMain)


# Plot the transformed data as a scatter plot. Recall that transforming
# the data will result in a NumPy NDArray. You can either use MatPlotLib
# to graph it directly, or you can convert it to DataFrame and have pandas
# do it for you.
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
ax.set_title('Unreduced Format of selectMain')
ax.set_xlabel('bgr')
ax.set_ylabel('rc')
ax.set_zlabel('wc')
ax.scatter(selectMain.bgr, selectMain.rc, selectMain.wc, c = 'green', marker = '.')
#

#plot reduced form of selectMain
fig = plt.figure()
ax2 = fig.add_subplot(111)
ax2.set_title('PCA reduction of Selected Features of KidneyData')
ax2.set_xlabel('Component 1')
ax2.set_ylabel('Component 2')
ax2.scatter(T[:, 0], T[:, 1], c = 'orange' , marker = 'o', alpha = 0.75)
plt.show()
# Since we've already demonstrated how to plot directly with MatPlotLib in
# Module4/assignment1.py, this time we'll convert to a Pandas Dataframe.
#
# Since we transformed via PCA, we no longer have column names. We know we
# are in P.C. space, so we'll just define the coordinates accordingly:
ax = helper.drawVectors(T, pca.components_, selectMain.columns.values, plt, scaleFeatures)
T = pd.DataFrame(T)
T.columns = ['component1', 'component2']

T.plot.scatter(x='component1', y='component2', marker='o', c=labels, alpha=0.75, ax=ax)
plt.show()



Overwriting assignment3.py
